In [1]:
import torchvision
from torchvision import models
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

import os, subprocess
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

In [2]:
data_dir = "./data"
dir_lst = os.listdir(data_dir)
for dir in dir_lst:
    dir_path = os.listdir(f"./data/{dir}")

    for name in dir_path:
        if name == ".ipynb_checkpoints":
            print(name)
            os.rmdir(f"./data/{dir}/{name}")

In [3]:
IMG_TRANSFORM = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((512,512), antialias = True),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5)),
    ])

In [4]:
train_imgfolder = ImageFolder(root="./data/train",
                              transform=IMG_TRANSFORM,
                              target_transform=None
                             )

test_imgfolder = ImageFolder(root="./data/test",
                              transform=IMG_TRANSFORM
                             )

In [5]:
Image.MAX_IMAGE_PIXELS = None

train_dataloader = DataLoader(dataset=train_imgfolder,
							  batch_size = 32,
                              num_workers=os.cpu_count(),
                              shuffle=True,
                              drop_last=True 
                             )

test_dataloader = DataLoader(dataset=test_imgfolder,
							  batch_size = 32,
                              num_workers=os.cpu_count(),
                              shuffle=True,
                              drop_last=True 
                            )

In [6]:
import torch
import torch.nn as nn

from tqdm import tqdm

pre_trained_model = models.resnet50(weights='ResNet50_Weights.IMAGENET1K_V1')
new_model = torch.nn.Sequential(*(list(pre_trained_model.children())[:-1]))


In [7]:
class modified_resnet50(nn.Module):
    def __init__(self, new_model):
        super(modified_resnet50,self).__init__()
        self.resnet50 = new_model
        self.fc = nn.Linear(2048, 2048)
        self.cls_fc = nn.Linear(2048,33)
        self.activation = nn.LeakyReLU()
        
    def forward(self, x):
        x = self.resnet50(x)
        x = x.squeeze(2,3)
        style_x = self.activation(self.fc(x))
        x = self.cls_fc(style_x)

        # # of classess, 2048
        return x, style_x     

In [8]:
# model setting
model = modified_resnet50(new_model)

EPOCH = 50
LEARNING_RATE = 1e-3 
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE, weight_decay = 0.01)
criterion = nn.CrossEntropyLoss()
print("Using device: ",DEVICE)

Using device:  cuda:0


In [9]:
def loss_fn(class_output, label, criterion):
    CE_loss = criterion(class_output, label)

    return CE_loss 

In [10]:
def train(train_loader, model, optimizer, criterion, epoch):
    model.train()
    correct = 0
    total = 0
    
    epoch_loss = []
    for index, (img_data,labels) in enumerate(train_loader):
        img_data, labels = img_data.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        
        class_output, encoded_vector = model(img_data)
        loss = loss_fn(class_output, labels, criterion)
        
        _, predicted = torch.max(class_output.data, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
        
        
        loss.backward()
        optimizer.step()

        epoch_loss.append(loss.item())
        if index%100==0:
            print(f'train loss : {np.mean(epoch_loss):>7f}, [epoch:{epoch}, iter:{index}]')
            
    accuracy = correct / total
    print(f"TRAIN ACCURACY : {(100*accuracy):>0.1f}% [{correct}/{total}]")
    return np.mean(epoch_loss)

In [11]:
def test(test_loader, model, criterion, epoch):
    model.eval()

    epoch_loss = []
    correct = 0
    total = 0
    
    with torch.no_grad():
        for index, (img_data,labels) in enumerate(test_loader):
            img_data, labels = img_data.to(DEVICE), labels.to(DEVICE)
            class_output, encoded_vector = model(img_data)
    
            loss = loss_fn(class_output, labels, criterion)

            _, predicted = torch.max(class_output.data, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
            
            epoch_loss.append(loss.item())
    
    accuracy = correct / total
    
    print(f"TEST ACCURACY : {(100*accuracy):>0.1f}% [{correct}/{total}]")
    return np.mean(epoch_loss)

In [14]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [13]:
train_loss_lst, test_loss_lst = [], []

model_version = 1


test_best_loss = 100
test_current_loss = 100
early_stop_threshold = 5
early_stop_trigger = 0

model = model.to(DEVICE)

for i in tqdm(range(EPOCH), desc = 'Train'):
    print(f"EPOCH {i+1} \n-------------------------")
    train_loss = train(train_dataloader, model, optimizer, criterion, i+1)
    test_loss = test(test_dataloader, model, criterion, i+1)

    train_loss_lst.append(train_loss)
    test_loss_lst.append(test_loss)

    if test_loss < test_best_loss :
        print("...MODEL SAVE...")
        test_best_loss = test_loss 
        
        if model_version == 0:
            torch.save(model.state_dict(),'contrastive_model_for_style_weight_v2_without_target_style.pth')
        else:
            torch.save(model.state_dict(),'contrastive_model_for_style_weight_resnet.pth')
        
    if test_current_loss < test_loss:
        early_stop_trigger += 1 
    else:
        early_stop_trigger = 0 
    test_current_loss  = test_loss 

    print(f'\nEPOCH:{i+1} | train loss : {train_loss}, test loss : {test_loss}\n')
    
    if early_stop_trigger >= early_stop_threshold:
        break
        
print("DONE!")

Train:   0%|                                       | 0/50 [00:00<?, ?it/s]

EPOCH 1 
-------------------------
train loss : 3.544906, [epoch:1, iter:0]
train loss : 3.297352, [epoch:1, iter:100]
train loss : 3.193942, [epoch:1, iter:200]
train loss : 3.163434, [epoch:1, iter:300]
train loss : 3.132053, [epoch:1, iter:400]
train loss : 3.113900, [epoch:1, iter:500]
train loss : 3.098014, [epoch:1, iter:600]
train loss : 3.086142, [epoch:1, iter:700]
train loss : 3.066486, [epoch:1, iter:800]
train loss : 3.052089, [epoch:1, iter:900]
train loss : 3.035867, [epoch:1, iter:1000]
train loss : 3.018818, [epoch:1, iter:1100]
train loss : 3.009904, [epoch:1, iter:1200]
train loss : 3.001343, [epoch:1, iter:1300]
train loss : 2.992356, [epoch:1, iter:1400]
train loss : 2.985194, [epoch:1, iter:1500]
train loss : 2.977347, [epoch:1, iter:1600]
train loss : 2.971984, [epoch:1, iter:1700]


/home/qkrwnstj/anaconda3/envs/qkrwnstj/lib/python3.9/site-packages/PIL/TiffImagePlugin.py:858: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))


train loss : 2.965426, [epoch:1, iter:1800]
train loss : 2.960060, [epoch:1, iter:1900]
train loss : 2.954554, [epoch:1, iter:2000]


/home/qkrwnstj/anaconda3/envs/qkrwnstj/lib/python3.9/site-packages/PIL/TiffImagePlugin.py:858: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))


...

train loss : 2.458933, [epoch:50, iter:1300]
train loss : 2.460763, [epoch:50, iter:1400]
train loss : 2.460239, [epoch:50, iter:1500]
train loss : 2.459692, [epoch:50, iter:1600]
train loss : 2.461168, [epoch:50, iter:1700]
train loss : 2.462873, [epoch:50, iter:1800]
train loss : 2.462800, [epoch:50, iter:1900]
train loss : 2.462316, [epoch:50, iter:2000]
train loss : 2.462488, [epoch:50, iter:2100]
train loss : 2.461086, [epoch:50, iter:2200]
train loss : 2.460640, [epoch:50, iter:2300]


/home/qkrwnstj/anaconda3/envs/qkrwnstj/lib/python3.9/site-packages/PIL/TiffImagePlugin.py:858: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))


train loss : 2.461306, [epoch:50, iter:2400]
train loss : 2.461136, [epoch:50, iter:2500]
TRAIN ACCURACY : 27.8% [22298/80128]


Train: 100%|█████████████████████████| 50/50 [15:30:25<00:00, 1116.50s/it]

TEST ACCURACY : 17.7% [3155/17824]

EPOCH:50 | train loss : 2.460976928900987, test loss : 2.8491460048531714

DONE!
